#### 1. IMPORT LIBRARIES AND CIFAR 10

In [0]:
# general
import numpy as np
import keras
import os
import time
import matplotlib.pyplot as plt
% matplotlib inline

# dataset
from keras.datasets import cifar10

# modeling tools
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.utils.np_utils import to_categorical

Check and Prepare Data:

In [36]:
# split data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# check data dimension
print('training data shape: {}'.format(x_train.shape))
print('test data shape: {}'.format(x_test.shape))
print('shape of one instance: {}'.format(x_train.shape[1:]))

training data shape: (50000, 32, 32, 3)
test data shape: (10000, 32, 32, 3)
shape of one instance: (32, 32, 3)


In [37]:
# check labels
labels = []
for y in y_train.flatten():
    if y not in labels:
        labels.append(y)
print('training labels are: {}'.format(labels))
print('# labels: {}'.format(len(labels)))

training labels are: [6, 9, 4, 1, 2, 7, 8, 3, 5, 0]
# labels: 10


In [0]:
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [0]:
# Convert class vectors to one-hot encoded vectors.
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

In [0]:
# # Transform images from (32,32,3) to 3072-dimensional vectors (32*32*3)

# x_train = np.reshape(x_train,(50000,3072))
# x_test = np.reshape(x_test,(10000,3072))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalization of pixel values (to [0-1] range)

x_train /= 255
x_test /= 255

#### 2. HELP FUNCTION

In [0]:
def plotAcc(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model_accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc = 'upper right')
    
def plotLoss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model_loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc = 'upper right')

#### 3. BUILD CNN CLASSIFIER

In [0]:
num_classes= 10

##### RMSprop - LR = 0.001

In [43]:
# hyper parameters to be tuned
n_neuron = 256
batch_size = 32
dropout_rate = 0.2
data_augmentation = True
epochs = 100
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# set up model structure
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
    
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              steps_per_epoch = int(50000/batch_size),
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
 
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    exp_optimizer_r = model.fit_generator(datagen.flow(x_train, y_train,
                                                   batch_size=batch_size),
                        epochs=epochs,
                        steps_per_epoch = int(50000/batch_size),
                        validation_data=(x_test, y_test),
                        workers=4)


# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/100
1562/1562 [==============================] - 46s 29ms/step - loss: 1.6273 - acc: 0.4086 - val_loss: 1.3811 - val_acc: 0.5069
Epoch 2/100
1562/1562 [==============================] - 45s 29ms/step - loss: 1.2908 - acc: 0.5421 - val_loss: 1.0818 - val_acc: 0.6185
Epoch 3/100
1562/1562 [==============================] - 45s 29ms/step - loss: 1.1752 - acc: 0.5871 - val_loss: 0.9303 - val_acc: 0.6809
Epoch 4/100
1562/1562 [==============================] - 45s 29ms/step - loss: 1.1369 - acc: 0.6088 - val_loss: 0.9409 - val_acc: 0.6673
Epoch 5/100
1562/1562 [==============================] - 45s 29ms/step - loss: 1.1373 - acc: 0.6133 - val_loss: 1.0834 - val_acc: 0.6434
Epoch 6/100
1562/1562 [==============================] - 44s 28ms/step - loss: 1.1492 - acc: 0.6145 - val_loss: 1.0103 - val_acc: 0.6676
Epoch 7/100
1562/1562 [==============================] - 44s 28ms/step - loss: 1.1831 - acc: 0.6056 - val_loss: 1.1338 - val_acc: 0.6397
Epoch 

##### Adam - LR = 0.001

In [45]:
# hyper parameters to be tuned
n_neuron = 256
batch_size = 32
dropout_rate = 0.2
data_augmentation = True
epochs = 100
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models_2')
model_name = 'keras_cifar10_trained_model_2.h5'

# set up model structure
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
 
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    exp_optimizer_a = model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        steps_per_epoch = int(50000/batch_size),
                        validation_data=(x_test, y_test),
                        workers=4)


# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/100
1562/1562 [==============================] - 48s 30ms/step - loss: 1.6336 - acc: 0.3956 - val_loss: 1.2100 - val_acc: 0.5694
Epoch 2/100
1562/1562 [==============================] - 46s 29ms/step - loss: 1.2844 - acc: 0.5405 - val_loss: 1.0644 - val_acc: 0.6191
Epoch 3/100
1562/1562 [==============================] - 46s 29ms/step - loss: 1.1445 - acc: 0.5922 - val_loss: 0.9082 - val_acc: 0.6828
Epoch 4/100
1562/1562 [==============================] - 45s 29ms/step - loss: 1.0591 - acc: 0.6245 - val_loss: 0.8969 - val_acc: 0.6835
Epoch 5/100
1562/1562 [==============================] - 46s 29ms/step - loss: 0.9990 - acc: 0.6496 - val_loss: 0.8265 - val_acc: 0.7131
Epoch 6/100
1562/1562 [==============================] - 46s 29ms/step - loss: 0.9601 - acc: 0.6628 - val_loss: 0.8198 - val_acc: 0.7087
Epoch 7/100
1562/1562 [==============================] - 46s 29ms/step - loss: 0.9342 - acc: 0.6730 - val_loss: 0.8232 - val_acc: 0.7103
Epoch 